In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
df1  = pd.read_csv('Pick_final.csv')
df1

,Unnamed: 0,Category,PickName
0,0,기타,"'News Sources', ""Ain't No Time For That. Groce..."
1,1,개발,'Creating a PreLaunch Landing Page that Conver...
2,2,AI,"'Data', 'Voice Command Innovations with AI', '..."
3,3,여행,"'Things To Do in Boston, MA', 'A weekend in Sa..."
4,4,맛집,"'NYC Food Instagrams', 'Awesome Restaurant', '..."
5,5,금융,"'Raising Capital in Boston', 'Taking Control o..."
6,6,디자인,"'원단이 필요할 때! 각종 원단 사이트 모음', 'Design Tips', '색다른..."
7,7,생활,"'Taking My Photography Beyond My Iphone', 'Str..."
8,8,사회,"'Journal', '아이겐코리아 보도자료', '반도체 산업 이해', '알고보면 쓸..."
9,9,게임,"'From Bronze to Grand Master: Overwatch', 'Bra..."


In [3]:
pick_vector= df1['PickName']
pick_vector

0     'News Sources', "Ain't No Time For That. Groce...
1     'Creating a PreLaunch Landing Page that Conver...
2     'Data', 'Voice Command Innovations with AI', '...
3     'Things To Do in Boston, MA', 'A weekend in Sa...
4     'NYC Food Instagrams', 'Awesome Restaurant', '...
5     'Raising Capital in Boston', 'Taking Control o...
6     '원단이 필요할 때! 각종 원단 사이트 모음', 'Design Tips', '색다른...
7     'Taking My Photography Beyond My Iphone', 'Str...
8     'Journal', '아이겐코리아 보도자료', '반도체 산업 이해', '알고보면 쓸...
9     'From Bronze to Grand Master: Overwatch', 'Bra...
10    'Training For A Marathon', "Women's World Cup ...
11    'Extra Virgin Olive Oil', '죽 끓이기', 'Vegan Rece...
12    'Pitch Perfect for Startups', 'Brand Creation ...
13    'Sending Out Your First Cold Marketing Email',...
14    'The Art of Surveys', 'Modern Classical Music ...
15    'Plant defense system', '식물', '여자쇼핑몰 싹 다 여기 집합...
16    '30대 한국인이 미국에 유학와서 영어로 살아남기', 'Summer Is Comin...
Name: PickName, dtype: object

In [4]:
# 텍스트 단어들의 어근 원형을 추출하기 위해 함수 생성

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import string
# string.puncitaion에 문자열의 모든 구두점이 들어있음
# 이를 활용해서 Tokenize시킬 때 구두점들을 제외하기 위한 것
# ord('문자열') => 문자열의 ASCII 코드를 반환해줌!
# duct(key, value)형태로 모든 구두점의 각 ASCII코드를 KEY값으로 넣어주자!

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

# 토큰화한 각 단어들의 원형들을 리스트로 담아서 반환
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]
# 텍스트를 Input으로 넣어서 토큰화시키고 토큰화된 단어들의 원형들을 리스트로 담아 반환

def LemNormalize(text):
    # translate인자에 구두점 dict넣어주어서 구두점 삭제해준 상태로 토큰화시키기
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# tf-idf 백터화시키면서 cusotmized해준 토큰화+어근추출 방식 tokenizer인자에 넣어주기
# 백터화시킬 tf-idf 도구 옵션 추가해서 구축
# 1,2gram적용, 빈도수 0.05이하, 0.85이상의 빈도수 단어들 제거
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
stop_words='english', ngram_range=(1,2), min_df=0.05, max_df=0.85)
# fit_transform으로 위해서 구축한 도구로 텍스트 백터화
ftr_vect = tfidf_vect.fit_transform(df1['PickName'])
ftr_vect

c:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


<17x20380 sparse matrix of type '<class 'numpy.float64'>'
	with 23467 stored elements in Compressed Sparse Row format>

In [5]:
from sklearn.cluster import KMeans
import time

word_vectors = ftr_vect
num_clusters = 17

# K means 를 정의하고 학습시킨다.
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )
df1['Category'] = idx

In [6]:
df1.loc[0]

Unnamed: 0                                                    0
Category                                                     15
PickName      'News Sources', "Ain't No Time For That. Groce...
Name: 0, dtype: object

In [7]:
# df1.rename(columns={'Unnamed: 0':"Labeling"}, inplace=True)
# df2 = df1.drop('Category', axis=1)
# df2
# 'ddfff.rename(index={0:'기타',1:'',2:'',3:'',4:'',5:'',6:'',7:'',8:'',9:'',10:'',11:'',12:'',13:'',14:'',15:'',16:''},inplace=True)

In [8]:
x = ftr_vect.toarray()
x.shape

(17, 20380)

In [9]:
from sklearn.manifold import MDS

y = MDS(n_components=2).fit_transform(x) # x: numpy.ndarray
y.shape

(17, 2)

In [10]:
import numpy as np

n_data_per_class = 50
n_classes = 3

x = []
y = []
for c in range(n_classes):
    x_ = 0.3 * np.random.random_sample((n_data_per_class, 2))
    x_ += np.random.random_sample((1, 2))
    x.append(x_)
    y.append(np.asarray([c] * n_data_per_class))
x = np.vstack(x)
y = np.concatenate(y)

In [11]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from bokeh.io import export_png

output_notebook()

title = '{} classes {} points'.format(n_classes, n_classes * n_data_per_class)

def draw_figure(x, y, title, show_figure=True,
    colors = 'firebrick darksalmon lightseagreen'.split()):

    p = figure(width=400, height=400, title=title)
    for c in range(n_classes):
        idx = np.where(y == c)[0]
        x_ = x[idx]
        p.scatter(x_[:,0], x_[:,1], fill_color=colors[c], line_color=colors[c])
    if show_figure:
        show(p)
    return p

p = draw_figure(x, y, title)

Loading BokehJS ...

In [12]:
from sklearn.manifold import LocallyLinearEmbedding

plots = []
for n_neighbors in [2, 3, 5, 10]:
    z = LocallyLinearEmbedding(
        n_components=2,
        n_neighbors=n_neighbors
    ).fit_transform(x)
    title = 'LLE with n_neighbors = {}'.format(n_neighbors)
    plots.append(draw_figure(z, y, title, show_figure=False))

gp = gridplot([[plots[0], plots[1]], [plots[2], plots[3]]])
show(gp)

In [70]:
from sklearn.manifold import Isomap

plots = []
for n_neighbors in [2, 3, 5, 10]:
    z = Isomap(
        n_components=10,
        n_neighbors=n_neighbors
    ).fit_transform(x)
    title = 'ISOMAP with n_neighbors = {}'.format(n_neighbors)
    plots.append(draw_figure(z, y, title, show_figure=False))
    
gp = gridplot([[plots[0], plots[1]], [plots[2], plots[3]]])
show(gp)

c:\Users\user\anaconda3\lib\site-packages\sklearn\manifold\_isomap.py:324: UserWarning: The number of connected components of the neighbors graph is 11 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
c:\Users\user\anaconda3\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
c:\Users\user\anaconda3\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
c:\Users\user\anaconda3\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
c:\Users\user\anaconda3\lib\si

In [78]:
from sklearn.manifold import TSNE

plots = []
for perplexity in [2, 5, 10, 30, 100, 200]:
    z = TSNE(
        n_components=2,
        perplexity=perplexity
    ).fit_transform(x)
    title = 't-SNE with perplexity = {}'.format(perplexity)
    plots.append(draw_figure(z, y, title, show_figure=False))

gp = gridplot([[plots[0], plots[1]], [plots[2], plots[3]], [plots[4], plots[5]]])
show(gp)

c:\Users\user\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarnin